In [1]:
!pip install google-api-python-client

'pip' is not recognized as an internal or external command,
operable program or batch file.


In [12]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import pandas as pd 
from datetime import datetime
import os 
from google_play_scraper import Sort, reviews_all
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload 


In [10]:
!pip install google_play_scraper 

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/50.2 kB ? eta -:--:--
     -------- ------------------------------- 10.2/50.2 kB ? eta -:--:--
     -------- ------------------------------- 10.2/50.2 kB ? eta -:--:--
     --------------- ----------------------- 20.5/50.2 kB 93.9 kB/s eta 0:00:01
     -------------------------------------- 50.2/50.2 kB 256.4 kB/s eta 0:00:00


In [13]:

# Define necessary constants
SERVICE_ACCOUNT_FILE = 'service_account.json'
SCOPES = ['https://www.googleapis.com/auth/drive']
PARENT_FOLDER_ID = '1yaX2B2xNmfUi_qTlmsPqDFHjGeTBf4Xw'  # Update with your parent folder ID

def authenticate():
    creds = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    return creds

def upload_data(file_path):
    creds = authenticate()
    service = build("drive", "v3", credentials=creds)

    file_metadata = {
        "app_store": "Jio",
        "parents": [PARENT_FOLDER_ID],  # Use 'parents' instead of 'path', and provide the folder ID
        "name": file_path.split("/")[-1]}  # Use the file name as the 'name' attribute
    
    
    media_body = MediaFileUpload(file_path)  # Import MediaFileUpload from googleapiclient.http

    file = service.files().create(
        body=file_metadata,
        media_body=media_body,
        fields='id'
    ).execute()

    print('File uploaded:', file.get('id'))

# Example usage
file_path = 'movies_db.xlsx'  # Update with the path to your file
upload_data(file_path)


File uploaded: 1bJ8uVvHdY0o3GVoPTU0zW50nR8-XFi8x


In [1]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import os
from google_play_scraper import Sort, reviews_all
from googleapiclient.http import MediaFileUpload

# Define necessary constants
SERVICE_ACCOUNT_FILE = 'service_account.json'
SCOPES = ['https://www.googleapis.com/auth/drive']
FOLDER_IDS = {
    "com.jio.media.jiobeats": "1xMbImCG8IXb1uLpZZ01-SgrwHHiqoj86",
    "com.spotify.music": "1FYVbhYihhJVjLBbhtHaTaH3WeeSD96GG",
    "com.bsbportal.music": "1UrTHLD0tOHQSe7-tO74Ua7CPoWuxqE2E"
}

def authenticate():
    creds = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    return creds

def upload_data(file_path, folder_id):
    creds = authenticate()
    service = build("drive", "v3", credentials=creds)

    media_body = MediaFileUpload(file_path)

    file_metadata = {
        'name': os.path.basename(file_path),
        'parents': [folder_id]
    }

    try:
        file = service.files().create(
            body=file_metadata,
            media_body=media_body,
            fields='id'
        ).execute()

        print('File uploaded:', file.get('id'))
    except Exception as e:
        print('An error occurred while uploading the file:', str(e))

app_id_lst = ["com.jio.media.jiobeats", "com.spotify.music", "com.bsbportal.music"]

for app_id in app_id_lst:
    result_all = []
    for _ in range(1, 15):
        result = reviews_all(
            app_id,
            sleep_milliseconds=0,
            lang='en',
            country='in',
            sort=Sort.NEWEST
        )
        result_all.extend(result)
    
    df = pd.DataFrame(result_all)
    df = df.drop_duplicates()
    print(df.shape)
    
    today = datetime.now().strftime("%m-%d-%Y_%H%M%S")
    folder_id = FOLDER_IDS.get(app_id)
    
    if folder_id:
        file_name = f'reviews-{app_id}_{today}.xlsx'
        file_path = os.path.join('data', file_name)
        df.to_excel(file_path, index=False)
        upload_data(file_path, folder_id)


(995, 11)
File uploaded: 1SI-_GRgd-va4ZikHEPVuWJKJTNuWc4pt
(995, 11)
File uploaded: 1LJ80fEV9Lj7hK7wf1lLQxQTgQXQmlEph
(1592, 11)
File uploaded: 1Od49Sedkp8PXVz0tUljVTbjreZ44Ov29
